In [38]:
import os
import glob
import numpy as np
import pandas as pd
import re
import dateparser
from datetime import timedelta

In [39]:
conjunto_dados = pd.DataFrame()

SPEAKER_REGEX = re.compile(r'[A|O] Sr\.ª?( [\w\-,]+)+([\-\w]+)?( \([\w\s\.,\-]+\))*(?=:)')
MULTILINE_SPEAKER_REGEX = re.compile("^" + SPEAKER_REGEX.pattern, flags=re.M) 

for file in os.scandir("./discursos"):
    speech_by_speaker = []

    with open(file) as fp:
        content = fp.read()

        data_sessao = re.search(r"\d+ de \w+ de \d{4}", content)


        #1) remover datas espalhadas, páginas e afins espalhadas pelos documentos (por causa da coversao PDF para TXT)

        datas_texto = re.compile("\d{1,2} [A-Z]+ [A-Z]+ [A-Z]+ \d{4} (\n\n\d+)?")
        datas_texto.sub(" ", content)

        series_numeros = re.compile("[A-Z]+ SÉRIE \W NÚMERO \d+ (\n+\d+)?")
        series_numeros.sub(" ", content)

        #2) criar dicionario de intervencoes para criar dataframe

        #2.1) identificar os intervenientes e intervencoes correspondentes

        deputadxs = MULTILINE_SPEAKER_REGEX.finditer(content) 
        deputadxs = [match.group() for match in deputadxs]
        deputadxs = list(set(deputadxs))

        #deputadxs = [deputadxs[5]]
        #print(deputadxs)

        content_on_one_line = ''.join(content.splitlines())

        for speaker in deputadxs:
            content_split_by_speaker = content_on_one_line.split(speaker + ':')
            # Get rid of everything before they speak
            content_split_by_speaker.pop(0)

            # what_they_say = [speech for speech in speaker_regex]
            # for each line in content_split_by_speaker
            # use the speaker regex to find the next speaker and get everything before that
            for line in content_split_by_speaker:
                start_of_next_speaker = SPEAKER_REGEX.finditer(line)

                # either an index or None
                # if it's None, either:
                # - the regex isn't accurate enough; or
                # - the speaker is the only person to speak]
                next_speaker_indexes = [match.start() for match in start_of_next_speaker]
                next_speaker_index = next_speaker_indexes[0] if len(next_speaker_indexes) > 0 else None

                speech = line[0:next_speaker_index] if next_speaker_index is not None else line
                speech_by_speaker.append((speaker, speech))

                # if "):" in speech:
                #     print(speech)
        
        df = pd.DataFrame.from_records(speech_by_speaker, columns=["speaker", "speech"])#.reset_index()
        df['filename'] = os.path.basename(file)
        df['date'] = data_sessao.group()
        #print(df)
                
        conjunto_dados = pd.concat([conjunto_dados, df])
            
conjunto_dados#.to_csv("deputados.csv")#, index=False)

,speaker,speech,filename,date
0,A Sr.ª Cecília Meireles (CDS-PP),"— Sr. Presidente, em primeiro lugar, quero cu...",darl13sl03n050.txt,22 de fevereiro de 2018
1,A Sr.ª Cecília Meireles (CDS-PP),"— Aliás, já tinha ouvido todas estas teorias ...",darl13sl03n050.txt,22 de fevereiro de 2018
2,A Sr.ª Cecília Meireles (CDS-PP),"— Portanto, torna-se particularmente difícil ...",darl13sl03n050.txt,22 de fevereiro de 2018
3,A Sr.ª Cecília Meireles (CDS-PP),— … porque são exatamente os mesmos momentos ...,darl13sl03n050.txt,22 de fevereiro de 2018
4,A Sr.ª Cecília Meireles (CDS-PP),"— … que nos levam a um fim que não é, de cert...",darl13sl03n050.txt,22 de fevereiro de 2018
...,...,...,...,...
229,A Sr.ª Gabriela Canavilhas (PS),— O Estado tem a obrigação de zelar por que o...,darl13sl03n058.txt,10 de março de 2018
230,A Sr.ª Gabriela Canavilhas (PS),"— Vou terminar, Sr. Presidente. Nesse sentido...",darl13sl03n058.txt,10 de março de 2018
231,A Sr.ª Gabriela Canavilhas (PS),— Errado!,darl13sl03n058.txt,10 de março de 2018
232,A Sr.ª Gabriela Canavilhas (PS),— Quem está em incumprimento é a Câmara Munic...,darl13sl03n058.txt,10 de março de 2018


In [40]:
conjunto_dados["speech"] = [string.lstrip("— ") for string in conjunto_dados["speech"]]


partidos_raw = conjunto_dados["speaker"].str.split("(", n=1, expand=True)
partidos_raw.columns = ["speaker", "partido"]
partidos_raw["partido"] = partidos_raw["partido"].str.rstrip(")")
partidos_raw["speech"] = conjunto_dados["speech"]
partidos_raw["filename"] = conjunto_dados["filename"]
partidos_raw["date"] = conjunto_dados["date"]
partidos_raw["speaker"] = partidos_raw["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")
partidos_raw["speaker"] = partidos_raw["speaker"].str.rstrip(" ")
#conjunto_dados = conjunto_dados.merge(partidos_raw, how="right", on="speaker") it was breaking (maybe memory issues?)

#partidos_raw.to_csv("deputados.csv", index=False)

/tmp/ipykernel_55198/1604839912.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  partidos_raw["speaker"] = partidos_raw["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")


In [41]:
#correções manuais partidos

partidos_raw.loc[partidos_raw["partido"] == "PEN", "partido"] = 'PAN'

partidos_raw.loc[partidos_raw["partido"] == "OS Verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os Vedes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os Verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "s Verdes", "partido"] = 'PEV'

partidos_raw.loc[partidos_raw["partido"] == "SD", "partido"] = 'PSD'
partidos_raw.loc[partidos_raw["partido"] == "PD", "partido"] = 'PSD'

partidos_raw.loc[partidos_raw["partido"] == "CDS", "partido"] = 'CDS-PP'
partidos_raw.loc[partidos_raw["partido"] == "CSD-PP", "partido"] = 'CDS-PP'
partidos_raw.loc[partidos_raw["partido"] == "CDS-P", "partido"] = 'CDS-PP'

partidos_raw.loc[partidos_raw["partido"] == "PC", "partido"] = 'PCP'

partidos_raw.loc[partidos_raw["partido"] == "B E", "partido"] = 'BE'
partidos_raw.loc[partidos_raw["partido"] == "Bloco de Esquerda", "partido"] = 'BE'

partidos_raw.loc[partidos_raw["partido"] == "Partido Socialista", "partido"] = 'PS'

partidos_raw.loc[partidos_raw["partido"] == "N insc.", "partido"] = "N insc"
partidos_raw.loc[partidos_raw["partido"] == "N Insc.", "partido"] = "N insc"
partidos_raw.loc[partidos_raw["partido"] == "Ninsc.", "partido"] = "N insc"
partidos_raw.loc[partidos_raw["partido"] == "N. insc.", "partido"] = "N insc"
partidos_raw.loc[partidos_raw["partido"] == "Não insc.", "partido"] = "N insc"

declaracoes_partido = partidos_raw.loc[(partidos_raw["partido"] == "PS") \
                |(partidos_raw["partido"] == "PSD") \
                |(partidos_raw["partido"] == "CDS-PP") \
                |(partidos_raw["partido"] == "BE") \
                |(partidos_raw["partido"] == "PCP")\
                |(partidos_raw["partido"] == "PEV")\
                |(partidos_raw["partido"] == "PAN")\
                |(partidos_raw["partido"] == "IL") \
                |(partidos_raw["partido"] == "CH") \
                |(partidos_raw["partido"] == "L") \
                |(partidos_raw["partido"] == "N insc")]

In [42]:
declaracoes_partido["new_date"] = [dateparser.parse(string, languages=["pt"]) for string in declaracoes_partido["date"]]
declaracoes_partido["new_date"] = declaracoes_partido["new_date"]- np.timedelta64(1,'D')
declaracoes_partido

/home/rute/anaconda3/lib/python3.8/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
/tmp/ipykernel_55198/2309180253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  declaracoes_partido["new_date"] = [dateparser.parse(string, languages=["pt"]) for string in declaracoes_partido["date"]]
/tmp/ipykernel_55198/2309180253.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,speaker,partido,speech,filename,date,new_date
0,Cecília Meireles,CDS-PP,"Sr. Presidente, em primeiro lugar, quero cumpr...",darl13sl03n050.txt,22 de fevereiro de 2018,2018-02-21
1,Cecília Meireles,CDS-PP,"Aliás, já tinha ouvido todas estas teorias qua...",darl13sl03n050.txt,22 de fevereiro de 2018,2018-02-21
2,Cecília Meireles,CDS-PP,"Portanto, torna-se particularmente difícil fin...",darl13sl03n050.txt,22 de fevereiro de 2018,2018-02-21
3,Cecília Meireles,CDS-PP,… porque são exatamente os mesmos momentos de ...,darl13sl03n050.txt,22 de fevereiro de 2018,2018-02-21
4,Cecília Meireles,CDS-PP,"… que nos levam a um fim que não é, de certeza...",darl13sl03n050.txt,22 de fevereiro de 2018,2018-02-21
...,...,...,...,...,...,...
229,Gabriela Canavilhas,PS,O Estado tem a obrigação de zelar por que o pr...,darl13sl03n058.txt,10 de março de 2018,2018-03-09
230,Gabriela Canavilhas,PS,"Vou terminar, Sr. Presidente. Nesse sentido, e...",darl13sl03n058.txt,10 de março de 2018,2018-03-09
231,Gabriela Canavilhas,PS,Errado!,darl13sl03n058.txt,10 de março de 2018,2018-03-09
232,Gabriela Canavilhas,PS,Quem está em incumprimento é a Câmara Municipa...,darl13sl03n058.txt,10 de março de 2018,2018-03-09


In [ ]:
declaracoes_partido["speech"] = [str(speech) for speech in declaracoes_partido["speech"]]


In [45]:
declaracoes_partido["gerigonca"] = declaracoes_partido["speech"].str.contains('gerigonça|gerigonças|geringonca|geringoncistas|geringonça|geringonçada|geringonçamente|geringonçar|geringonças', regex=True)

/tmp/ipykernel_55198/1929719215.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  declaracoes_partido["gerigonca"] = declaracoes_partido["speech"].str.contains('gerigonça|gerigonças|geringonca|geringoncistas|geringonça|geringonçada|geringonçamente|geringonçar|geringonças', regex=True)


In [91]:
gerigonca = declaracoes_partido.loc[declaracoes_partido["gerigonca"] == True]
gerigonca.to_csv("gerigonca_raw.csv")
gerigonca = gerigonca.groupby(["new_date","partido"])["speech"].count().transpose().unstack().fillna(0)
gerigonca = gerigonca.astype(int)
gerigonca.to_csv("gerigonca.csv")

In [84]:
gerigonca = gerigonca.reset_index()

In [86]:
gerigonca

partido,new_date,BE,CDS-PP,CH,IL,PAN,PCP,PEV,PS,PSD
0,2015-11-18,0,1,0,0,0,0,0,0,0
1,2015-11-27,0,0,0,0,0,0,0,1,0
2,2015-12-03,0,2,0,0,0,0,0,0,0
3,2015-12-09,0,1,0,0,0,1,0,0,0
4,2015-12-16,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
225,2021-11-17,0,1,0,1,0,0,0,0,0
226,2021-11-19,0,2,0,0,0,0,0,0,0
227,2021-11-24,0,0,0,0,0,0,0,0,1
228,2021-11-26,0,1,0,0,0,0,0,0,0


In [88]:
gerigonca.groupby(gerigonca["new_date"].dt.to_period('m')).sum().to_csv("gerigonca.csv")

In [89]:
gerigonca.groupby(gerigonca["new_date"].dt.to_period('y')).sum().to_csv("gerigonca-year.csv")